In [0]:
from confluent_kafka import Producer
import json
from  itertools import islice
import numpy as np
from pyspark.sql.functions import col, decode, split, element_at,udf
import logging
from pyspark.sql.types import StructType, StructField, StringType
from pyspark import SparkContext
import datetime

In [0]:



eh_kv_secret = dbutils.secrets.get(scope="ingest00-meta002-sbox", key="evh-namespace-connection-string")

# Event Hub configurations
eventhubs_hostname = "sbox-dlrm-eventhub-ns.servicebus.windows.net:9093"
conf = {
    'bootstrap.servers': eventhubs_hostname,
    'security.protocol': 'SASL_SSL',
    'sasl.mechanism': 'PLAIN',
    'sasl.username': '$ConnectionString',
    'sasl.password': eh_kv_secret ,
    'retries': 5,                     # Increased retries
    'enable.idempotence': True,        # Enable idempotent producer
}
broadcast_conf = sc.broadcast(conf)



In [0]:
# Read and prepare data HTML files
json_mount = '/mnt/ingest00curatedsboxgold/ARIADM/ARM/JOH/'
binary_df = spark.read.format('binaryFile') \
                     .option('pathGlobFilter', '*.a360') \
                     .option('recursiveFileLookup', 'true') \
                     .load(json_mount)



html_df = binary_df.withColumn("content_str", decode(col('content'), 'utf-8')) \
                   .withColumn('file_path', element_at(split(col('path'), '/'), -1))
html_df = html_df.select('content_str','file_path')

# # Repartition based on cluster resources
# num_spark_partitions =  8
# optimized_html_df = html_df.repartition(num_spark_partitions)

html_df.display()

## Send to EventHubs

In [0]:
# Repartition based on cluster resources
num_spark_partitions =  8
optimized_html_df = html_df.repartition(num_spark_partitions)

In [0]:
def process_partition(partition):
    import logging
    from confluent_kafka import Producer
    from datetime import datetime

    # Initialize logger
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger('KafkaProducer')
    
    failure_list = []
    success_list = []
    results = []

    # Initialize producer
    producer = Producer(**broadcast_conf.value)

    def delivery_report(err, msg):
        key_str = msg.key().decode('utf-8') if msg.key() is not None else "Unknown"
        if err is not None:
            err_msg = str(err)
            logger.error(f"Message delivery failed for key {key_str}: {err}")
            failure_list.append((key_str, "failure", err_msg, datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")))
        else:
            success_list.append((key_str, "success", "", datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")))

    for row in partition:
        if row.file_path is None or row.content_str is None:
            logger.warning(f"Skipping row with missing file_path/content_str: {row}")
            continue

        try:
            if isinstance(row.content_str, str):
                value = row.content_str.encode('utf-8')
            elif isinstance(row.content_str, bytearray):
                value = bytes(row.content_str)
            elif isinstance(row.content_str, bytes):
                value = row.content_str
            else:
                logger.error(f"Unsupported type for content_str: {type(row.content_str)}")
                failure_list.append((row.file_path, "failure", "Unsupported type", datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")))
                continue

            producer.produce(
                topic='evh-joh-pub-dev-uks-dlrm-01',
                key=row.file_path.encode('utf-8'),
                value=value,
                callback=delivery_report
            )

        except BufferError:
            logger.error("Producer buffer full. Polling for events.")
            producer.poll(1)  
        except Exception as e:
            logger.error(f"Unexpected error during production: {e}")
            failure_list.append((row.file_path, "failure", str(e), datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")))

    try:
        producer.flush()
        logger.info("Producer flushed successfully.")
    except Exception as e:
        logger.error(f"Unexpected error during flush: {e}")

    # Append results to list instead of using yield
    results.extend(success_list)
    results.extend(failure_list)

    return results  # Return list instead of using yield

# Schema for result DataFrame
schema = StructType([
    StructField("file_name", StringType(), True),
    StructField("status", StringType(), True),
    StructField("error_message", StringType(), True),
    StructField("timestamp", StringType(), True)
])

# Apply the optimized processing
result_rdd = optimized_html_df.rdd.mapPartitions(process_partition)

# Create DataFrame and show results
result_df = spark.createDataFrame(result_rdd, schema)
display(result_df)  # Debugging step to verify output

In [0]:
# # Display failed files

failed_files = result_df.filter(col("status") == "failure")

display(failed_files)
failed_files.count()

In [0]:
result_df.write.format("delta").mode("append").save("/mnt/ingest00curatedsboxgold/ARIADM/ARM/AUDIT/JOH/JOH_pub_audit_db_eh")